In [ ]:
import pandas as pd
from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, Alignment
from google.colab import files
import io
import os

# Upload Excel file
print("📂 Please upload your student Excel file:")
uploaded = files.upload()

# Read the uploaded file
for name in uploaded:
    df = pd.read_excel(io.BytesIO(uploaded[name]))

# Filter only eligible students
df = df[df['Eligibility'] == 'Eligible']

# Group by Year and Branch
grouped = df.groupby(['Year', 'Branch']).agg(
    Demand_Students=('USN', 'count'),
    Demand_Amount=('Demand', 'sum'),
    Collection_Students=('Amount Paid', lambda x: (x > 0).sum()),
    Collection_Amount=('Amount Paid', 'sum'),
    Balance_Students=('Balance', lambda x: (x > 0).sum()),
    Balance_Amount=('Balance', 'sum')
).reset_index()

output_path = "Formatted_Student_Balance_Report.xlsx"

if os.path.exists(output_path):
    # Load existing workbook
    wb = load_workbook(output_path)

    # Delete existing 'ABSTRACT' sheet if it exists
    if 'ABSTRACT' in wb.sheetnames:
        std = wb['ABSTRACT']
        wb.remove(std)

    # Create new 'ABSTRACT' sheet
    ws = wb.create_sheet('ABSTRACT')
else:
    # Create new workbook and sheet
    wb = Workbook()
    ws = wb.active
    ws.title = "ABSTRACT"

# Institute heading
ws.merge_cells('B1:H1')
ws['B1'] = "RAMAIAH INSTITUTE OF TECHNOLOGY, BANGALORE - 560054"
ws['B1'].font = Font(bold=True, size=14)
ws['B1'].alignment = Alignment(horizontal='center')

# Sub-heading
ws.merge_cells('B2:H2')
ws['B2'] = "(Autonomous Institute, affiliated to VTU, Belgaum)"
ws['B2'].alignment = Alignment(horizontal='center')

# Report title
ws.merge_cells('B3:H3')
ws['B3'] = "Balance Report Receivable ( Eligible and Registered Students )"
ws['B3'].alignment = Alignment(horizontal='center')

# Academic year
ws.merge_cells('B5:H5')
ws['B5'] = "ACADEMIC YEAR 2021-22"
ws['B5'].alignment = Alignment(horizontal='center')
ws['B5'].font = Font(bold=True)

# Main headers
ws.append(['Year', 'BRANCH',
           'NO OF STUDENTS', 'AMOUNT',
           'NO OF STUDENTS', 'AMOUNT',
           'NO OF STUDENTS', 'AMOUNT'])

# Section headers
ws.merge_cells('C6:D6')
ws['C6'] = 'DEMAND'
ws.merge_cells('E6:F6')
ws['E6'] = 'COLLECTION'
ws.merge_cells('G6:H6')
ws['G6'] = 'BALANCE'
for cell in ['C6', 'E6', 'G6']:
    ws[cell].alignment = Alignment(horizontal='center')
    ws[cell].font = Font(bold=True)

# Insert grouped data
for _, row in grouped.iterrows():
    ws.append([
        row['Year'],
        row['Branch'],
        row['Demand_Students'], row['Demand_Amount'],
        row['Collection_Students'], row['Collection_Amount'],
        row['Balance_Students'], row['Balance_Amount']
    ])

# Save the workbook (overwrite existing file)
wb.save(output_path)

# Download the updated file
files.download(output_path)
